<a href="https://colab.research.google.com/github/Rachhh53/ExploreWiki/blob/main/WikipediaApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs



In [1]:
!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-py3-none-any.whl size=13476 sha256=13b581577a2af6b6b6119de7a546977ca659e0aef05a351868877950ef48cdc5
  Stored in directory: /root/.cache/pip/wheels/d3/24/56/58ba93cf78be162451144e7a9889603f437976ef1ae7013d04
Successfully built wikipedia-api


In [5]:
!pip install tqdm

# Imports

In [6]:
# https://towardsdatascience.com/auto-generated-knowledge-graphs-92ca99a81121
import wikipediaapi  # pip install wikipedia-api
import pandas as pd
import concurrent.futures
from tqdm import tqdm

# Run

In [3]:
def wiki_scrape(topic_name, verbose=True):
    def wiki_link(link):
        try:
            page = wiki_api.page(link)
            if page.exists():
                return {'page': link, 'text': page.text, 'link': page.fullurl,
                        'categories': list(page.categories.keys())}
        except:
            return None

    wiki_api = wikipediaapi.Wikipedia(language='en',
                                      extract_format=wikipediaapi.ExtractFormat.WIKI)
    page_name = wiki_api.page(topic_name)
    if not page_name.exists():
        print('Page {} does not exist.'.format(topic_name))
        return

    page_links = list(page_name.links.keys())
    progress = tqdm(desc='Links Scraped', unit='', total=len(page_links)) if verbose else None
    sources = [{'page': topic_name, 'text': page_name.text, 'link': page_name.fullurl,
                'categories': list(page_name.categories.keys())}]

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_link = {executor.submit(wiki_link, link): link for link in page_links}
        for future in concurrent.futures.as_completed(future_link):
            data = future.result()
            sources.append(data) if data else None
            progress.update(1) if verbose else None
    progress.close() if verbose else None

    namespaces = ('Wikipedia', 'Special', 'Talk', 'LyricWiki', 'File', 'MediaWiki',
                  'Template', 'Help', 'User', 'Category talk', 'Portal talk')
    sources = pd.DataFrame(sources)
    sources = sources[(len(sources['text']) > 20)
                      & ~(sources['page'].str.startswith(namespaces, na=True))]
    sources['categories'] = sources.categories.apply(lambda x: [y[9:] for y in x])
    sources['topic'] = topic_name
    print('Wikipedia pages scraped:', len(sources))

    return sources

In [7]:
wiki_data = wiki_scrape('Nestle')

Links Scraped: 100%|██████████| 761/761 [00:09<00:00, 79.07/s] 


Wikipedia pages scraped: 164


In [12]:
wiki_data

,page,text,link,categories,topic
0,Nestle,Nestlé S.A. (; French: [nɛsle]) is a Swiss mul...,https://en.wikipedia.org/wiki/Nestl%C3%A9,"[1866 establishments in Switzerland, All Wikip...",Nestle
1,1812 N Moore,1812 North Moore is an office building in Arli...,https://en.wikipedia.org/wiki/1812_N_Moore,"[2013 establishments in Virginia, Buildings an...",Nestle
2,20th Century Press Archives,The 20th Century Press Archives (German: Press...,https://en.wikipedia.org/wiki/20th_Century_Pre...,"[AC with 0 elements, Archives in Germany, Arti...",Nestle
3,ABB,"ABB Ltd (German: ABB AG, French, Italian, Roma...",https://en.wikipedia.org/wiki/ABB,"[ABB, Articles containing German-language text...",Nestle
4,2022 Russian invasion of Ukraine,"Russia invaded Ukraine on 24 February 2022, ma...",https://en.wikipedia.org/wiki/2022_Russian_inv...,"[2022 Russian invasion of Ukraine, 2022 contro...",Nestle
...,...,...,...,...,...
159,Criticisms of Cargill,This article addresses various criticisms of C...,https://en.wikipedia.org/wiki/Criticisms_of_Ca...,"[All articles with dead external links, Articl...",Nestle
160,Criticism of fast food,Criticism of fast food includes claims of nega...,https://en.wikipedia.org/wiki/Criticism_of_fas...,"[All articles needing expert attention, All ar...",Nestle
161,Criticism of McDonald's,McDonald's is an American multinational fast f...,https://en.wikipedia.org/wiki/McDonald%27s,"[1940 establishments in California, 1940 estab...",Nestle
162,Crosse & Blackwell,Crosse & Blackwell is a British food brand. Th...,https://en.wikipedia.org/wiki/Crosse_%26_Black...,"[1706 establishments in England, Articles with...",Nestle


In [13]:
# Get Page Sections

def print_sections(sections, level=0):
        for s in sections:
                print("%s: %s - %s" % ("*" * (level + 1), s.title, s.text[0:40]))
                print_sections(s.sections, level + 1)

topic_name = 'Nestle'
wiki_api = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI)
page_name = wiki_api.page(topic_name)
print_sections(page_name.sections)

*: History - 
**: 1866–1900: Founding and early years - Nestlé's origin dates back to the 1860s,
***: Timeline - 1866, Charles Page (US consul to Switzer
**: 1901–1989: Mergers - In 1904, François-Louis Cailler, Charles
**: 1990–2011: Growth internationally - The first half of the 1990s proved to be
**: 2012–present: Recent developments - In recent years, Nestlé Health Science h
*: Corporate affairs and governance - Nestlé is the biggest food company in th
**: Financial data - 
**: Joint ventures - Joint ventures include:

Cereal Partners
**: Board of Directors - As of 2017, the board is composed of:
Pa
**: Lobbying - The company engages third party lobbying
*: Products - Nestlé currently has over 2,000 brands w
**: Food safety - 
***: Milk products and baby food - In late September 2008, the Hong Kong go
***: Cookie dough - In June 2009, an outbreak of E. coli O15
***: Maggi noodles - In May 2015, Food Safety Regulators from
*: Sponsorships - 
**: Music and Entertainment - In 1993, pl

In [14]:
!pip install wptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 KB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycurl: filename=pycurl-7.45.1-cp37-cp37m-linux_x86_64.whl size=300958 sha256=deaf66084bc715eb9c9d7390b27450d9cf2a1a31b522783a8333517c4ef2b088
  Stored in directory: /root/.cache/pip/wheels/b4/fe/1f/e6cae4153dbc082271bd5763ad36ae255639e4641054db10bc
Successfully built pycurl


# wptools - doesn't working on my machine

In [33]:
import wptools
so = wptools.page('Nestle').get_parse()
infobox = so.data['infobox']
infobox

en.wikipedia.org (parse) Nestle
en.wikipedia.org (imageinfo) File:NestleHQ.jpg
Nestlé (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:NestleHQ....
  infobox: <dict(24)> name, former_names, logo, image, image_size,...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:N...
  pageid: 160227
  parsetree: <str(193316)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Nestlé
  wikibase: Q160746
  wikidata_url: https://www.wikidata.org/wiki/Q160746
  wikitext: <str(164424)> {{Short description|Swiss multinational ...
}


{'name': 'Nestlé S.A.',
 'former_names': '{{collapsible list|\n* Anglo-Swiss Condensed Milk Company (1866–1867)\n* Farine Lactée Henri Nestlé (1867–1905)\n* Nestlé and Anglo-Swiss Condensed Milk Company (1905–1947)\n* Nestlé Alimentana SA (1947–)}}',
 'logo': 'Nestlé.svg',
 'image': 'NestleHQ.jpg',
 'image_size': '250px',
 'image_caption': 'Nestlé Headquarters in [[Vevey]], [[Vaud]], Switzerland',
 'type': '[[Public company|Public]]',
 'traded_as': '{{SWX|NESN}}',
 'ISIN': 'CH0038863350',
 'foundation': '{{start date and age|df|=|yes|1866}}',
 'founder': '[[Henri Nestlé]]',
 'location': '[[Vevey]], [[Vaud]], [[Switzerland]]',
 'area_served': 'Worldwide',
 'key_people': '[[Paul Bulcke]] <br> {{small|([[Chairman]])}} <br>[[Ulf Mark Schneider]] <br> {{small|([[CEO]])}} <br/>David McDaniel <br/> {{small|([[CFO]])}}',
 'industry': '[[Food processing]]',
 'products': '[[Baby food]], [[coffee]], [[dairy products]], [[breakfast cereals]], [[confectionery]], [[bottled water]], [[ice cream]], [[

# beautiful soup way

In [17]:
!pip install bs4

In [24]:
# importing the module
import requests
import bs4

# URL
URL = "https://en.wikipedia.org/wiki/Twitter"

# sending the request
response = requests.get(URL)

# parsing the response
soup = bs4.BeautifulSoup(response.text, 'html')

# Now, we have parsed HTML with us. I want to get the _motto_ from the wikipedia page.
# Elements structure
# table - class="infobox"
# 3rd tr to get motto

# getting infobox
infobox = soup.find('table', {'class': 'infobox'})
infobox


# # getting 3rd row element tr
# third_tr = infobox.find_all('tr')[2]

# # from third_tr we have to find first 'a' element and 'div' element to get required data
# first_a = third_tr.div.find('a')
# div = third_tr.div.div

# # motto
# motto = f"{first_a.text} {div.text[:len(div.text) - 3]}"

# # printing the motto
# print(motto)


<table class="infobox vcard"><caption class="infobox-title fn org">Twitter, Inc.</caption><tbody><tr><td class="infobox-image" colspan="2"><a class="image" href="/wiki/File:Twitter-logo.svg"><img alt="Twitter-logo.svg" data-file-height="421" data-file-width="512" decoding="async" height="82" src="//upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Twitter-logo.svg/100px-Twitter-logo.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Twitter-logo.svg/150px-Twitter-logo.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Twitter-logo.svg/200px-Twitter-logo.svg.png 2x" width="100"/></a></td></tr><tr><td class="infobox-image" colspan="2"><div class="mw-collapsible mw-collapsed" style="box-sizing:border-box;width:100%;font-size:95%;padding:4px;border:none;"><div style="font-size:100%;line-height:1.6;font-weight:bold;background:transparent;text-align:left;;background:gainsboro;text-align:center">Screenshot</div><div class="mw-collapsible-content" style="backgr

# Pandas html way ***

In [41]:
from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/Nestle'
infoboxes = read_html(page, index_col=0, attrs={"class":"infobox"})
wikitables = read_html(page, index_col=0, attrs={"class":"wikitable"})

print("Extracted {num} infoboxes".format(num=len(infoboxes)))
print("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 infoboxes
Extracted 1 wikitables


In [58]:
wikitables[0]

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Year,,,,,,,,,,,
Revenue,83.642,92.186,92.158,91.612,88.785,89.469,89.791,91.439,92.568,84.343,87.088
Net income,9.487,10.611,10.015,14.456,9.066,8.531,7.183,10.135,12.609,12.232,17.196
Assets,114.091,126.229,120.442,133.450,123.992,131.901,130.380,137.015,127.940,124.028,139.142
Employees,328000.000,339000.000,333000.000,339000.000,335000.000,328000.000,323000.000,308000.000,291000.000,273000.000,276000.000


In [50]:
infobox_df = infoboxes[0]
infobox_df

,1
0,
NaN,NaN
"Nestlé Headquarters in Vevey, Vaud, Switzerland","Nestlé Headquarters in Vevey, Vaud, Switzerland"
Formerly,List Anglo-Swiss Condensed Milk Company (1866–...
Type,Public
Traded as,SIX: NESN
ISIN,CH0038863350
Industry,Food processing
Founded,1866; 156 years ago
Founder,Henri Nestlé


In [63]:
founded = infobox_df.loc["Founded"]
industry = infobox_df.loc["Industry"]
num_employees = infobox_df.loc["Number of employees"]

In [65]:
founded

1    1866; 156 years ago
Name: Founded, dtype: object

In [66]:
industry

1    Food processing
Name: Industry, dtype: object

In [67]:
num_employees

1    276,000 (2021)[3]
Name: Number of employees, dtype: object